<a href="https://colab.research.google.com/github/etoshey/colabC3D/blob/main/ScaleTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Opensim Sclaing

In [ ]:
#@title Install MiniConda (~40s)
import sys
import os

# ensure cwd is /content
os.chdir("/content")

# Install MiniConda
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local

# Add the path to the conda package installation location
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [ ]:
#@title Install opensim with Conda (~1min)

# Install Opensim
!conda install -c opensim-org opensim -y
import opensim as osim

In [1]:
#@title Importing Misc Libraries
from google.colab import files #to upload user's files
import numpy as np #used in some functions
import time
!pip install xmltodict


In [3]:
#@title Step1 : upload static trc file
import ipywidgets as widgets
from google.colab import files

file_name = ""

upload_btn = widgets.Button(description='Upload')
def upload_btn_eventhandler(obj):

  global file_name
  uploaded = files.upload()
  file_name = list(uploaded.keys())[0]

  print('Upload Completed ==> '+file_name)


upload_btn.on_click(upload_btn_eventhandler)
display(upload_btn)



Button(description='Upload', style=ButtonStyle())

Saving acq3 prova_4f8.trc to acq3 prova_4f8.trc
Upload Completed ==> acq3 prova_4f8.trc


In [71]:
#@title OpenSim SclaeTool Form
import os
import ipywidgets as widgets
import xmltodict


static_markers = []
#get all files
all_xml_files =[]
all_osim_files =[]
all_trc_files =['select..']
for file_path in os.listdir('/content/'):
    if os.path.isfile(os.path.join('/content/', file_path)):
      if os.path.splitext(file_path)[1]=='.xml':
        all_xml_files.append(file_path)
      elif os.path.splitext(file_path)[1]=='.osim':
        all_osim_files.append(file_path)
      elif os.path.splitext(file_path)[1]=='.trc':
        all_trc_files.append(file_path)



#--------------------------------------------------------------------------------------------------------#
all_trc_dropdown = widgets.Dropdown(description='Selecte static data(trc):',options = all_trc_files)
select_trc_button = widgets.Button(
    description='OK',
    button_style='info'
)

def dropdown_handler(change):
    global static_trc_path
    static_trc_path = change.new
all_trc_dropdown.observe(dropdown_handler, names='value')


def select_trc_btn_eventhandler(obj):
  global static_markers,static_trc_path
  my_file = open(static_trc_path, "r")
  my_file.readline();#line1
  my_file.readline();#line2
  my_file.readline();#line3
  param = my_file.readline();
  static_markers = param.split('\t')
  static_markers = [item for item in static_markers if item != '']
  static_markers = static_markers[2:]
  show_form()

select_trc_button.on_click(select_trc_btn_eventhandler)

#--------------------------------------------------------------------------------------------------------#

h0 = widgets.HBox([all_trc_dropdown, select_trc_button ])
display(h0)

#--------------------------------------------------------------------------------------------------------#


def show_form():
  #-------> Section 1
  section1_lbl =  widgets.Label(value="----------------------------- Anthropometry & Model -----------------------------");
  mass = widgets.Text(value='76',placeholder='Type something',description='Weight(Kg):',disabled=False)
  height = widgets.Text(value='1800',placeholder='Type something',description='height(Cm):',disabled=False)

  #-------> GenericModelMaker
  model_dropdown = widgets.Dropdown(description='Model:',options=['gait2392_simbody', 'gait2354_simbody','gait10dof18musc'])
    #Generate markerSet.xml based on static trc

  #-------> ModelScaler
  ModelScaler_lbl =  widgets.Label(value="----------------------------- ModelScaler -----------------------------");
  # torso
  torso1_marker_dropdown = widgets.Dropdown(description='Torso 1:',options = static_markers)
  torso2_marker_dropdown = widgets.Dropdown(description='Torso 2:',options = static_markers)
  torso3_marker_dropdown = widgets.Dropdown(description='Torso 3:',options = static_markers)
  #pelvis
  pelvis1_marker_dropdown = widgets.Dropdown(description='Pelvis 1:',options = static_markers)
  pelvis2_marker_dropdown = widgets.Dropdown(description='Pelvis 2:',options = static_markers)
  #thigh
  Lthigh1_marker_dropdown = widgets.Dropdown(description='Left thigh1:',options = static_markers)
  Lthigh2_marker_dropdown = widgets.Dropdown(description='Left thigh2:',options = static_markers)
  Rthigh1_marker_dropdown = widgets.Dropdown(description='Right thigh1:',options = static_markers)
  Rthigh2_marker_dropdown = widgets.Dropdown(description='Right thigh2:',options = static_markers)
  #shank
  Lshank1_marker_dropdown = widgets.Dropdown(description='Left shank1:',options = static_markers)
  Lshank2_marker_dropdown = widgets.Dropdown(description='Left shank2:',options = static_markers)
  Rshank1_marker_dropdown = widgets.Dropdown(description='Right shank1:',options = static_markers)
  Rshank2_marker_dropdown = widgets.Dropdown(description='Right shank2:',options = static_markers)
  #foot
  Lfoot1_marker_dropdown = widgets.Dropdown(description='Left foot1:',options = static_markers)
  Lfoot2_marker_dropdown = widgets.Dropdown(description='Left foot2:',options = static_markers)
  Rfoot1_marker_dropdown = widgets.Dropdown(description='Right foot1:',options = static_markers)
  Rfoot2_marker_dropdown = widgets.Dropdown(description='Right foot2:',options = static_markers)



  spliter_lbl =  widgets.Label(value="---------------------------------------------------------------------");
  #marker_file
  marker_file = widgets.Label(value="static trial : " + static_trc_path)
  #MarkerSet
  MarkerSet_dropdown = widgets.Dropdown(description='MarkerSet :',options = ['Plug-in Gait' , 'Cast' , 'Cleveland'])
  MarkerSet_dropdown.observe(dropdown_MarkerSet, names='value')

  spliter_lbl2 =  widgets.Label(value="---------------------------------------------------------------------");


  #Processing
  run_button = widgets.Button(description='Run!',button_style='success')
  # display


  h1 = widgets.HBox([mass, height , model_dropdown])
  h2 = widgets.HBox([torso1_marker_dropdown,torso2_marker_dropdown,torso3_marker_dropdown])
  h3 = widgets.HBox([pelvis1_marker_dropdown,pelvis2_marker_dropdown])
  h4 = widgets.HBox([Lthigh1_marker_dropdown,Lthigh2_marker_dropdown,Rthigh1_marker_dropdown,Rthigh2_marker_dropdown])
  h5 = widgets.HBox([Lshank1_marker_dropdown,Lshank2_marker_dropdown,Rshank1_marker_dropdown,Rshank2_marker_dropdown])
  h6 = widgets.HBox([Lfoot1_marker_dropdown,Lfoot2_marker_dropdown,Rfoot1_marker_dropdown,Rfoot2_marker_dropdown])
  h7 = widgets.VBox([marker_file,MarkerSet_dropdown])



  display(section1_lbl,h1,ModelScaler_lbl,h2,h3,h4,h5,h6,spliter_lbl,h7,spliter_lbl2)




def Setup_Xml_Generator():
  print("ss")



def dropdown_MarkerSet(change):
  global static_trc_path

  xml_file_name = ""
  match change.new:
    case "Cleveland":
      if os.path.isfile("/content/gait2392_Scale_MarkerSet.xml") == False:
        !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/MarkerSet/gait2392_Scale_MarkerSet.xml
      xml_file_name = "gait2392_Scale_MarkerSet.xml"

  print(xml_file_name)
  if xml_file_name != "":
    #read xml file
    xml_data = open(xml_file_name, 'r').read()  # Read data
    xmlDict = xmltodict.parse(xml_data)
    M_list = xmlDict['OpenSimDocument']['MarkerSet']['objects']['Marker']
    list_marker_name = []
    _list1 = []
    _list2 = []
    _list3 = []
    _list4 = []

    for i in range(0, len(M_list),4):
      list_marker_name.append(m['@name'])
      if (len(M_list) - i >= 4):
        _list1.append(widgets.Dropdown(description=M_list[i]['@name'] +':',options = static_markers))
        _list2.append(widgets.Dropdown(description=M_list[i+1]['@name'] +':',options = static_markers))
        _list3.append(widgets.Dropdown(description=M_list[i+2]['@name'] +':',options = static_markers))
        _list4.append(widgets.Dropdown(description=M_list[i+3]['@name'] +':',options = static_markers))

    # make table

    V1 = widgets.VBox(_list1)
    V2 = widgets.VBox(_list2)
    V3 = widgets.VBox(_list3)
    V4 = widgets.VBox(_list4)
    H = widgets.HBox([V1,V2,V3,V4])
    Title =  widgets.Label(value="Label Matching ==>");
    display(Title,H)






Label(value='----------------------------- Anthropometry & Model -----------------------------')

Label(value='----------------------------- ModelScaler -----------------------------')

Label(value='---------------------------------------------------------------------')

Label(value='---------------------------------------------------------------------')

gait2392_Scale_MarkerSet.xml


Label(value='Label Matching ==>')

In [36]:
from lxml import etree

# xml_data = open('gait2392_Scale_MarkerSet.xml', 'r').read()  # Read data



# print(xml_data)


xml_data = open('gait2392_Scale_MarkerSet.xml', 'r').read()  # Read data
xmlDict = xmltodict.parse(xml_data)
M_list = xmlDict['OpenSimDocument']['MarkerSet']['objects']['Marker']
list_marker_name = []
for m in M_list:
  list_marker_name.append(m['@name'])

print(list_marker_name)





['Sternum', 'R.Acromium', 'L.Acromium', 'Top.Head', 'R.ASIS', 'L.ASIS', 'V.Sacral', 'R.Thigh.Upper', 'R.Thigh.Front', 'R.Thigh.Rear', 'R.Knee.Lat', 'R.Knee.Med', 'R.Shank.Upper', 'R.Shank.Front', 'R.Shank.Rear', 'R.Ankle.Lat', 'R.Ankle.Med', 'R.Heel', 'R.Midfoot.Sup', 'R.Midfoot.Lat', 'R.Toe.Lat', 'R.Toe.Med', 'R.Toe.Tip', 'L.Thigh.Upper', 'L.Thigh.Front', 'L.Thigh.Rear', 'L.Knee.Lat', 'L.Knee.Med', 'L.Shank.Upper', 'L.Shank.Front', 'L.Shank.Rear', 'L.Ankle.Lat', 'L.Ankle.Med', 'L.Heel', 'L.Midfoot.Sup', 'L.Midfoot.Lat', 'L.Toe.Lat', 'L.Toe.Med', 'L.Toe.Tip']


In [70]:
!wget -q  https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/MarkerSet/gait2392_Scale_MarkerSet.xml